In [1]:
import sys
import argparse
import os
import torch
from collections import OrderedDict
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import seaborn as sns
import pylab as py
import time
from pyDOE import lhs
import warnings
sys.path.insert(0, '../../../Scripts/')
from models_imperfect import Generator, Discriminator, Q_Net
from cGAN import *
# from ../Scripts/helper import *

warnings.filterwarnings('ignore')

np.random.seed(1234)

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda:3')
else:
    device = torch.device('cpu')

## Hyper-parameters

In [3]:
num_epochs = 10000
lambda_val = 0
lambda_q = 0.5
tr_frac = 0.4

#architecture for the models
d_hid_dim = 50 
d_num_layer = 2

g_hid_dim = 50
g_num_layer = 4

q_hid_dim = 50
q_num_layer = 3

### Load Data

In [4]:
data = np.loadtxt('../../../datasets/tossing_trajectories.txt')
x = data[:, 0:6]
labels = data[:, 6:]

# training and test split
n_obs = int(tr_frac * x.shape[0])
train_x , train_y = x[:n_obs,:] , labels[:n_obs, :] 
test_x , test_y = x[n_obs:,:] , labels[n_obs:, :]

noise_dim = 2
data_dim = train_x.shape[1]
out_dim = labels.shape[1]

#### Models

In [5]:
D = Discriminator(in_dim = (data_dim + out_dim), out_dim = 1, hid_dim=d_hid_dim, num_layers=d_num_layer).to(device)
G = Generator(in_dim = (noise_dim + data_dim), out_dim = out_dim, hid_dim=g_hid_dim, num_layers=g_num_layer).to(device)
Q = Q_Net(in_dim=(out_dim + data_dim), out_dim=noise_dim, hid_dim=q_hid_dim, num_layers=q_num_layer).to(device)

In [6]:
cGAN = Tossing_cGAN(train_x, train_y, test_x, test_y, G, D, Q, device, num_epochs, lambda_val, noise_dim)

### Training

In [7]:
cGAN.train()

[Epoch 0/10000] [MSE loss: 4.313746] [G loss: 2.585699] [D loss: 5.459814] [Q loss: 3.931562] [Phy loss: 446.808578] [Adv G loss: 0.507475]
[Epoch 100/10000] [MSE loss: 0.326428] [G loss: 1.146808] [D loss: 5.304407] [Q loss: 0.933784] [Phy loss: 44.076407] [Adv G loss: 0.779352]
[Epoch 200/10000] [MSE loss: 0.414620] [G loss: 1.504537] [D loss: 5.396243] [Q loss: 0.580448] [Phy loss: 43.932301] [Adv G loss: 1.330773]
[Epoch 300/10000] [MSE loss: 0.488610] [G loss: 3.156284] [D loss: 4.435032] [Q loss: 0.402177] [Phy loss: 37.386539] [Adv G loss: 3.018759]
[Epoch 400/10000] [MSE loss: 0.309339] [G loss: 6.245313] [D loss: 4.039273] [Q loss: 0.328128] [Phy loss: 40.285904] [Adv G loss: 6.157700]
[Epoch 500/10000] [MSE loss: 0.306792] [G loss: 8.331911] [D loss: 3.512184] [Q loss: 0.351914] [Phy loss: 39.721399] [Adv G loss: 8.179800]
[Epoch 600/10000] [MSE loss: 0.242978] [G loss: 8.685697] [D loss: 4.915272] [Q loss: 0.144462] [Phy loss: 36.303698] [Adv G loss: 8.595669]
[Epoch 700/100

In [8]:
nsamples = 500
train_predictions_list = []
for run in range(nsamples):
    G_train_noise = cGAN.sample_noise(cGAN.train_x.shape[0], cGAN.noise_dim).to(device)
    train_predictions = cGAN.G.forward(torch.cat([cGAN.train_x, G_train_noise], dim=1)).detach().cpu().numpy()
    train_predictions = (train_predictions * cGAN.Ystd) + cGAN.Ymean
    train_predictions_list.append(train_predictions)

train_predictions = np.mean(train_predictions_list, axis=0)
train_predictions_dev = np.var(train_predictions_list, axis=0)
train_predictions = torch.Tensor(train_predictions).float().to(device)

std_y_t = torch.Tensor(cGAN.Ystd).float().to(device)
mean_y_t = torch.Tensor(cGAN.Ymean).float().to(device)
std_x_t = torch.Tensor(cGAN.Xstd).float().to(device)
mean_x_t = torch.Tensor(cGAN.Xmean).float().to(device)

train_y = (cGAN.train_y * std_y_t) + mean_y_t
train_x = (cGAN.train_x * std_x_t) + mean_x_t

mse_train = ((train_predictions - train_y)**2).mean()
phy_train = torch.mean(cGAN.physics_loss(train_x, train_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Training Set : ", mse_train.detach().cpu().numpy())
print("Physical Consistency: ", phy_train)

MSE on Training Set :  0.40761125
Physical Consistency:  1.183105


In [9]:
test_predictions_list = []
for run in range(nsamples):
    G_test_noise = cGAN.sample_noise(cGAN.test_x.shape[0], cGAN.noise_dim).to(device)
    test_predictions = cGAN.G.forward(torch.cat([cGAN.test_x, G_test_noise], dim=1)).detach().cpu().numpy()
    test_predictions = (test_predictions * cGAN.Ystd) + cGAN.Ymean
    test_predictions_list.append(test_predictions)

test_predictions = np.mean(test_predictions_list, axis=0)
test_predictions_dev = np.var(test_predictions_list, axis=0)
test_predictions = torch.Tensor(test_predictions).float().to(device)

test_y = (cGAN.test_y * std_y_t) + mean_y_t
test_x = (cGAN.test_x * std_x_t) + mean_x_t

mse_test = ((test_predictions - test_y)**2).mean()
phy_test = torch.mean(cGAN.physics_loss(test_x, test_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Test Set : ", mse_test.detach().cpu().numpy())
print("Physical Consistency: ", phy_test)

MSE on Test Set :  0.7447767
Physical Consistency:  1.778059
